In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import datetime
import pprint
import os

In [3]:
uri = os.getenv("MONGODB")
client = MongoClient(uri, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected!


In [4]:
db = client.sample_airbnb.listingsAndReviews

In [6]:
db.find_one()['price']

Decimal128('135.00')

In [7]:
db.find_one()

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [9]:
db.find_one()['cleaning_fee']

Decimal128('135.00')

In [23]:
db.find_one({"property_type":"House"},{"address.street": 1,"price":1})

{'_id': '16253247',
 'price': Decimal128('194.00'),
 'address': {'street': 'Kailua-Kona, HI, United States'}}

In [25]:
db.find_one({"price":{"$gt":500}},{
     "listing_url":1, 
     "name":1,
     "host_name":1, 
     "host_location":1, 
     "reviewer_name":1,
     "price":1
})

{'_id': '10030955',
 'listing_url': 'https://www.airbnb.com/rooms/10030955',
 'name': 'Apt Linda Vista Lagoa - Rio',
 'price': Decimal128('701.00')}

In [27]:
db.find_one(
     {
        "address.country":"Brazil",
        "review_scores.review_scores_rating":{"$gt":9}
    },
    {
        "name":1,
        "address":1,
        "review_scores_rating":1
    }
)


{'_id': '10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'address': {'street': 'Rio de Janeiro, Rio de Janeiro, Brazil',
  'suburb': 'Catete',
  'government_area': 'Catete',
  'market': 'Rio De Janeiro',
  'country': 'Brazil',
  'country_code': 'BR',
  'location': {'type': 'Point',
   'coordinates': [-43.18015675229857, -22.92638234778768],
   'is_location_exact': True}}}

In [34]:
db.find_one(
    {
        "amenities":"Hot tub",
        "address.country":"United States",
        "name":{"$exists":True},
        "address":{"$exists":True},
        "reviews.reviewer_name":{"$exists":True},
        "review_scores.review_scores_rating":{"$exists":True}
    }
)

{'_id': '4069429',
 'listing_url': 'https://www.airbnb.com/rooms/4069429',
 'name': '#Private Studio - Waikiki Dream',
 'summary': 'At beginning Waikiki, the tourist center of Honolulu, this charming studio is about 10min walking distance from the beach, and all happenings.  Across the bridge to Convention Center and 15min walk to Ala Moana mall. Bus line is across the street. Studio is fully furnished - full size bed,TV, Wifi, mid size fridge, full bath, microwave, coffee maker, induction hot plate. Building has laundry, pool, jacuzzi and BBQ area; for additional fee you may use gym and parking ($25/24hrs in-out privileges)',
 'space': 'Perfect location. Designed to be lived in on long term, this studio is suitable for long and short stays. Ocean view. Walking distance to the beach,  restaurants, shops, …',
 'description': "At beginning Waikiki, the tourist center of Honolulu, this charming studio is about 10min walking distance from the beach, and all happenings.  Across the bridge t

In [38]:
db.find_one({
    "price":{"$gte":200,"$lte":400},
    "amenities":"Pool"
},
{
    "name":1,
    "amenities":1,
    "price":1
})

{'_id': '10166986',
 'name': 'Resort-like living in Williamsburg',
 'amenities': ['TV',
  'Internet',
  'Wifi',
  'Air conditioning',
  'Wheelchair accessible',
  'Pool',
  'Kitchen',
  'Doorman',
  'Gym',
  'Elevator',
  'Hot tub',
  'Buzzer/wireless intercom',
  'Heating',
  'Family/kid friendly',
  'Washer',
  'Dryer',
  'Smoke detector',
  'Essentials',
  '24-hour check-in',
  'Hangers',
  'Iron'],
 'price': Decimal128('220.00')}

In [39]:
db.find_one({
    "amenities":"Pool",
    "$or":[
        {"address.country":"Canada"},
        {"address.country":"Mexico"}
    ]
    
})

{'_id': '12757761',
 'listing_url': 'https://www.airbnb.com/rooms/12757761',
 'name': 'Renovated 2 Bedroom/2 Bathroom Apartment P0305',
 'summary': 'All of our apartments include free wireless internet, air conditioning/heating, satellite TV, a phone with free US/Canada calls, one outdoor parking space, and weekly cleaning services. Extra rollaway bed available!',
 'space': 'The unit is fully equipped to make you feel at home. The apartment has a balcony, a living room, 2 bedrooms and 2 bathrooms, a dining area, and a fully equipped kitchen!',
 'description': 'All of our apartments include free wireless internet, air conditioning/heating, satellite TV, a phone with free US/Canada calls, one outdoor parking space, and weekly cleaning services. Extra rollaway bed available! The unit is fully equipped to make you feel at home. The apartment has a balcony, a living room, 2 bedrooms and 2 bathrooms, a dining area, and a fully equipped kitchen! Guests have access to the laundry room, garden,

In [43]:
list(db.aggregate([
    {
        "$group":{
            "_id":{
                "listing_url":"$listing_url",
                "name":"$name",
                "country":"$country",
                "review_scores":"$review_scores"
            },
            "count":{"$sum":1}},
    },
    {   "$sort":{"count":-1}
    },
    {
        "$limit":10
    },
    {
        "$project":{
            "_id":0,
            "name":"$_id.name",
            "country":"$_id.country",
            "review_scores":"$_id.review_scores",
            "count":1
        }
    }
    
]))

[{'count': 1,
  'name': '【城遇】樱空 日式loft公寓 近罗湖口岸国贸金光华万象城东门 双地铁口 好住好逛又好吃',
  'review_scores': {'review_scores_accuracy': 10,
   'review_scores_cleanliness': 9,
   'review_scores_checkin': 10,
   'review_scores_communication': 10,
   'review_scores_location': 10,
   'review_scores_value': 9,
   'review_scores_rating': 95}},
 {'count': 1,
  'name': 'Beautiful House, Views, Single King, Own Bathroom',
  'review_scores': {'review_scores_accuracy': 10,
   'review_scores_cleanliness': 10,
   'review_scores_checkin': 10,
   'review_scores_communication': 10,
   'review_scores_location': 9,
   'review_scores_value': 10,
   'review_scores_rating': 98}},
 {'count': 1,
  'name': 'Bilgi Üniversitesi yanında güzel ev',
  'review_scores': {'review_scores_accuracy': 10,
   'review_scores_cleanliness': 10,
   'review_scores_checkin': 10,
   'review_scores_communication': 10,
   'review_scores_location': 9,
   'review_scores_value': 10,
   'review_scores_rating': 95}},
 {'count': 1,
  'name': 'Sunny and q

In [50]:
db.find_one(
        {"amenities":"Essentials",
        "review_scores.review_scores_rating":{"$gte":8}
        },
        {
            "listing_url": 1,
            "name": 1,
            "address": 1,
            "review_scores": 1,
            "_id": 0
        }
    

)

{'listing_url': 'https://www.airbnb.com/rooms/10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'address': {'street': 'Rio de Janeiro, Rio de Janeiro, Brazil',
  'suburb': 'Catete',
  'government_area': 'Catete',
  'market': 'Rio De Janeiro',
  'country': 'Brazil',
  'country_code': 'BR',
  'location': {'type': 'Point',
   'coordinates': [-43.18015675229857, -22.92638234778768],
   'is_location_exact': True}},
 'review_scores': {'review_scores_accuracy': 10,
  'review_scores_cleanliness': 6,
  'review_scores_checkin': 8,
  'review_scores_communication': 8,
  'review_scores_location': 8,
  'review_scores_value': 8,
  'review_scores_rating': 80}}

In [55]:
list(db.find(
    {
        "price":{"$exists":True}
        
    }
).sort("price",-1).limit(1))

[{'_id': '20275354',
  'listing_url': 'https://www.airbnb.com/rooms/20275354',
  'name': 'İstanbul un kalbi sisli. Center of istanbul sisli',
  'summary': 'We are new married couple.We have have one fully furnished single room for rent in our flat which is located in Sisli Bomonti. The room is fully furnished, recently painted and large enough for one person to live in comfortably. The flat is large (130 m2), recently painted, has a big balcony and fully furnished with 24/7 hot water, limitless wireless',
  'space': 'our flat which is located in Sisli Bomonti just two minutes from the central bus stops going to Taksim, Besiktas, Levent, 5minutes from Sisli Metro Station, 7minutes from Mecidiyekoy Square and 10minutes to nisantasi. The room is fully furnished, recently painted and large enough for one person to live in comfortably. The flat is large (130 m2), recently painted, has a big balcony and fully furnished with 24/7 hot water, limitless wireless',
  'description': 'We are new ma

In [60]:
db.find_one({"host.host_is_superhost":True})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [61]:
db.find_one({
    "address.location.coordinates":{"$ne":"null"}
})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [63]:
db.find_one({
    "beds":{"$gte":2}
})

{'_id': '10051164',
 'listing_url': 'https://www.airbnb.com/rooms/10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'summary': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'space': 'Este quarto está preparado para acomodar confortavelmente quatro pessoas, podendo ainda ser acrescentado um sofá-cama para receber mais um hóspede.',
 'description': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'neighborhood_overview': 'A Grande Morada situa-se no Bairro do Catete, histórico bairro ladeado pelo morro de Santa Tereza, pelo bairro da Lapa, pelo Parque do Flamengo e pelo Lar

In [66]:
db.find_one({
    "host.host_name":{"$regex":"Livia"}
})

{'_id': '10030955',
 'listing_url': 'https://www.airbnb.com/rooms/10030955',
 'name': 'Apt Linda Vista Lagoa - Rio',
 'summary': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'space': '',
 'description': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Apartment',
 'room_type': 'Private room',
 'bed_type': 'Real Bed',
 'minimum_nights': '1',
 'maximum_nights': '1125',
 'cancellation_policy': 'flexible',
 'last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
 'calendar_last_

In [68]:
db.find_one({
    "host.host_location":"Brazil"
})

{'_id': '10324377',
 'listing_url': 'https://www.airbnb.com/rooms/10324377',
 'name': 'Suíte em local tranquilo e seguro',
 'summary': 'Casa próxima à Barra em Jacarepaguá, com piscina, churrasqueira e internet. Facilidade de deslocamento à Barra e Centro da Cidade.',
 'space': '',
 'description': 'Casa próxima à Barra em Jacarepaguá, com piscina, churrasqueira e internet. Facilidade de deslocamento à Barra e Centro da Cidade.',
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'House',
 'room_type': 'Private room',
 'bed_type': 'Real Bed',
 'minimum_nights': '1',
 'maximum_nights': '1125',
 'cancellation_policy': 'moderate',
 'last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
 'calendar_last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
 'accommodates': 1,
 'bedrooms': 1,
 'beds': 1,
 'number_of_reviews': 0,
 'bathrooms': Decimal128('1.0'),
 'amenities': ['Cable TV',
  'Internet',
  'Wifi',
  'Pool

In [69]:
db.find_one({
    "availability.availability_365":{"$gt":200}
})

{'_id': '10051164',
 'listing_url': 'https://www.airbnb.com/rooms/10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'summary': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'space': 'Este quarto está preparado para acomodar confortavelmente quatro pessoas, podendo ainda ser acrescentado um sofá-cama para receber mais um hóspede.',
 'description': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'neighborhood_overview': 'A Grande Morada situa-se no Bairro do Catete, histórico bairro ladeado pelo morro de Santa Tereza, pelo bairro da Lapa, pelo Parque do Flamengo e pelo Lar

In [70]:
db.find_one({
    #ne = not equal
    "cleaning_fee":{"$ne":"null"}
})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [72]:
db.find_one({
    "$and":[
        {"amenities":"TV"},
        {"amenities":"Wifi"}
    ]
})

db.find_one({
    "amenities":
        {"$all":["TV","Wifi"]},
    
})

{'_id': '10051164',
 'listing_url': 'https://www.airbnb.com/rooms/10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'summary': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'space': 'Este quarto está preparado para acomodar confortavelmente quatro pessoas, podendo ainda ser acrescentado um sofá-cama para receber mais um hóspede.',
 'description': "Old floor, intirely reformed, destined to receive people in a very harmonious and welcoming place. Located at Catete's neighborhood, with a very easy access to the metro station, a lot of bus lines, next to a excellent trade and things to do around.",
 'neighborhood_overview': 'A Grande Morada situa-se no Bairro do Catete, histórico bairro ladeado pelo morro de Santa Tereza, pelo bairro da Lapa, pelo Parque do Flamengo e pelo Lar

In [73]:
db.find_one({
    "host.host_verifications":"jumio",
    "host.about":{"$ne":""}
})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [75]:
db.find_one({
    "bathrooms":{"$gte":2}
},
{
     "listing_ur": 1,
     "name" : 1,
     "property_type" : 1,
      "bed": 1,
    "bathrooms": 1,
      "price": 1
})

{'_id': '10051164',
 'name': "Catete's Colonial Big Hause Room B",
 'property_type': 'House',
 'bathrooms': Decimal128('4.0'),
 'price': Decimal128('250.00')}

In [77]:
db.find_one({
    "price":{"$gte":500},
    "security_deposit":{"$gte":1000}
})

{'_id': '10030955',
 'listing_url': 'https://www.airbnb.com/rooms/10030955',
 'name': 'Apt Linda Vista Lagoa - Rio',
 'summary': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'space': '',
 'description': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Apartment',
 'room_type': 'Private room',
 'bed_type': 'Real Bed',
 'minimum_nights': '1',
 'maximum_nights': '1125',
 'cancellation_policy': 'flexible',
 'last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
 'calendar_last_

In [79]:
db.find_one({
    "reviews":{
        "$exists":True,
        # reviews is a sub-document, $size can check the length of the doc
        "$not":{"$size":0}
    }
})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [83]:
db.find_one({
    "address.suburb":{"$regex":"Lagoa"}
})

{'_id': '10030955',
 'listing_url': 'https://www.airbnb.com/rooms/10030955',
 'name': 'Apt Linda Vista Lagoa - Rio',
 'summary': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'space': '',
 'description': 'Quarto com vista para a Lagoa Rodrigo de Freitas, cartão postal do Rio de Janeiro. Linda Vista.  1 Quarto e 1 banheiro  Amplo, arejado, vaga na garagem. Prédio com piscina, sauna e playground.  Fácil acesso, próximo da praia e shoppings.',
 'neighborhood_overview': '',
 'notes': '',
 'transit': '',
 'access': '',
 'interaction': '',
 'house_rules': '',
 'property_type': 'Apartment',
 'room_type': 'Private room',
 'bed_type': 'Real Bed',
 'minimum_nights': '1',
 'maximum_nights': '1125',
 'cancellation_policy': 'flexible',
 'last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
 'calendar_last_

In [85]:
db.find_one({
    "host.host_is_superhost":True,
    "host.host_total_listings_count":{"$gt":1}
})

{'_id': '1003530',
 'listing_url': 'https://www.airbnb.com/rooms/1003530',
 'name': 'New York City - Upper West Side Apt',
 'summary': '',
 'space': 'Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement.',
 'description': "Murphy bed, optional second bedroom available. Wifi available, Hulu, Netflix, TV Eat-in kitchen. Bathroom with great shower/bath.  Washer/dryer in basement. New York City! Great neighborhood - many terrific restaurants, bakeries, bagelries. Within easy walking distance are restaurants with the cuisines from India, Thailand, Japan, China, Mexico, South America and Europe.  As well as the many small independent stores that line Broadway, there chain stores such as Urban Outfitters (clothing), Whole Foods (groceries), Sephora (cosmetics), Michaels (crafts), and Modell's (sporting goods). Equidistant to Central Park and Riverside Park which have walking/running/biking t

In [93]:
list(db.aggregate([
    {
        "$project":{
            "listing_url":1,
            "name":1,
            # "address":1,
            "host_verification":"$host.host_verifications",
            "host_verification_count":{"$size":"$host.host_verifications"}
        }
    },
    {
        "$limit":3
    }
]))

[{'_id': '1003530',
  'listing_url': 'https://www.airbnb.com/rooms/1003530',
  'name': 'New York City - Upper West Side Apt',
  'host_verification': ['email',
   'phone',
   'reviews',
   'jumio',
   'offline_government_id',
   'government_id'],
  'host_verification_count': 6},
 {'_id': '10051164',
  'listing_url': 'https://www.airbnb.com/rooms/10051164',
  'name': "Catete's Colonial Big Hause Room B",
  'host_verification': ['email',
   'phone',
   'facebook',
   'reviews',
   'jumio',
   'government_id'],
  'host_verification_count': 6},
 {'_id': '10057447',
  'listing_url': 'https://www.airbnb.com/rooms/10057447',
  'name': 'Modern Spacious 1 Bedroom Loft',
  'host_verification': ['email',
   'phone',
   'jumio',
   'offline_government_id',
   'selfie',
   'government_id',
   'identity_manual'],
  'host_verification_count': 7}]

In [95]:
db.find_one({
    "host.host_verifications":{"$size":3}
})

{'_id': '10009999',
 'listing_url': 'https://www.airbnb.com/rooms/10009999',
 'name': 'Horto flat with small garden',
 'summary': 'One bedroom + sofa-bed in quiet and bucolic neighbourhood right next to the Botanical Garden. Small garden, outside shower, well equipped kitchen and bathroom with shower and tub. Easy for transport with many restaurants and basic facilities in the area.',
 'space': 'Lovely one bedroom + sofa-bed in the living room, perfect for two but fits up to four comfortably.  There´s a small outside garden with a shower There´s a well equipped open kitchen with both 110V / 220V wall plugs and one bathroom with shower, tub and even a sauna machine! All newly refurbished!',
 'description': 'One bedroom + sofa-bed in quiet and bucolic neighbourhood right next to the Botanical Garden. Small garden, outside shower, well equipped kitchen and bathroom with shower and tub. Easy for transport with many restaurants and basic facilities in the area. Lovely one bedroom + sofa-bed

In [100]:
list(db.aggregate([
    {
        "$match":{
            "host.host_verifications":{"$exists":True},
            "$expr":{
                "$gte": [{"$size":"$host.host_verifications"}, 3]
            }
        }
    },
    {
        "$project":{
            "listing_url":1,
            "name":1,
            "address":1,
            "host.host_verifications":1,
            "host_verifications_count":{"$size":"$host.host_verifications"}
        }
    },
    {
        "$limit":1
    }
]))

[{'_id': '1003530',
  'listing_url': 'https://www.airbnb.com/rooms/1003530',
  'name': 'New York City - Upper West Side Apt',
  'host': {'host_verifications': ['email',
    'phone',
    'reviews',
    'jumio',
    'offline_government_id',
    'government_id']},
  'address': {'street': 'New York, NY, United States',
   'suburb': 'Manhattan',
   'government_area': 'Upper West Side',
   'market': 'New York',
   'country': 'United States',
   'country_code': 'US',
   'location': {'type': 'Point',
    'coordinates': [-73.96523, 40.79962],
    'is_location_exact': False}},
  'host_verifications_count': 6}]